In [1]:
import pandas as pd
import time

In [2]:
data = pd.read_csv('movie_bd_v5.csv', encoding='utf-8')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 206.7+ KB


In [4]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [5]:
answers = {}  # создадим словарь для ответов


def string_to_list(series):
    '''Распарсить строку из нескольких значений и сохранить в список'''
    series = series.str.split('|')
    return series


def get_answer_form_df(df):
    '''Напечатать название фильма в нужном формате, если он один в датафрейме'''
    string = df.original_title.iloc[0] + ' (' + str(df.imdb_id.iloc[0]) + ')'
    return string


def pivot_table_from_data(df, row, column, value):
    '''Собираем сводную таблицу, где на вход передаётся df = DataFrame, 
    row = индекс новой таблицы, собранный из столбца df.row
    column - столбцы новой таблицы. собранные из строк в df.column 
    value - значения, соответствующие df.value
    '''
    table = pd.DataFrame(index=list(df[row]))

    for index in df.index:
        for i, col in enumerate(df.loc[index][column]):
            if col in table.columns:
                table[col][df.loc[index][row]] = df.loc[index][value]
            else:
                table[col] = pd.Series(dtype=float)
                table[col][df.loc[index][row]] = df.loc[index][value]
    return table


month_dict = {
    1: 'Январь',
    2: 'Февраль',
    3: 'Март',
    4: 'Апрель',
    5: 'Май',
    6: 'Июнь',
    7: 'Июль',
    8: 'Август',
    9: 'Сентябрь',
    10: 'Октябрь',
    11: 'Ноябрь',
    12: 'Декабрь'
}

def get_month(s):
    return time.strptime(s, '%m/%d/%Y').tm_mon

def get_season(s):
    '''Определяем сезон по номеру месяца'''
    month = time.strptime(s, '%m/%d/%Y').tm_mon
    if month in (1,2,12):
        season = 'Зима'
    elif month in (3,4,5):
        season = 'Весна'
    elif month in (6,7,8):
        season = 'Лето'
    else:
        season = 'Осень'
    return season



In [6]:
# преобразуем ячейки столбцы, где значений записаны в строку, в списки

new_data = data.copy()

new_data[['cast', 'genres', 'director', 'production_companies']] = \
    new_data[['cast', 'genres', 'director', 'production_companies']].apply(string_to_list)

new_data['profit'] = new_data['revenue'] - new_data['budget']

In [7]:
new_data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit
1726,tt0427327,50000000,90450008,Hairspray,"[John Travolta, Michelle Pfeiffer, Christopher...",[Adam Shankman],"When you follow your own beat, the world will ...","Pleasantly plump teenager, Tracy Turnblad and ...",117,"[Comedy, Music, Romance]",[New Line Cinema],7/13/2007,6.4,2007,40450008
377,tt0892769,165000000,494878759,How to Train Your Dragon,"[Jay Baruchel, Gerard Butler, Craig Ferguson, ...","[Dean DeBlois, Chris Sanders]",One adventure will change two worlds,As the son of a Viking leader on the cusp of m...,98,"[Fantasy, Adventure, Animation, Family]","[DreamWorks Animation, Vertigo Entertainment, ...",3/5/2010,7.4,2010,329878759
805,tt1772240,5000000,25562924,Apollo 18,"[Ryan Robbins, Warren Christie, Lloyd Owen, An...",[Gonzalo LÃ³pez-Gallego],There's a reason we've never gone back to the ...,"Officially, Apollo 17 was the last manned miss...",86,"[Horror, Thriller, Science Fiction]","[Bazelevs Production, Dimension Films, Bekmamb...",7/20/2011,4.9,2011,20562924
1772,tt0762114,35000000,69307224,License to Wed,"[Robin Williams, Mandy Moore, John Krasinski, ...",[Ken Kwapis],First came love... then came Reverend Frank.,"Newly engaged, Ben and Sadie can't wait to sta...",91,[Comedy],"[Village Roadshow Pictures, Warner Bros. Pictu...",7/4/2007,5.1,2007,34307224
1808,tt0207201,70000000,374111707,What Women Want,"[Helen Hunt, Mel Gibson, Marisa Tomei, Lauren ...",[Nancy Meyers],He has the power to hear everything women are ...,Advertising executive Nick Marshall is as cock...,127,"[Comedy, Romance]","[Paramount Pictures, Centropolis Entertainment...",12/14/2000,6.0,2000,304111707


# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [8]:
answer_1 = data[data.budget == data.budget.max()]

# в словарь вставляем номер вопроса и ваш ответ на него
answers['1'] = get_answer_form_df(answer_1)

print(answers['1'])

Pirates of the Caribbean: On Stranger Tides (tt1298650)


ВАРИАНТ 2

In [9]:
# можно добавлять разные варианты решения

In [10]:
answer_1_v2 = data.loc[data.budget.idxmax()]

print('{} ({})'.format(answer_1_v2['original_title'], answer_1_v2['imdb_id']))

Pirates of the Caribbean: On Stranger Tides (tt1298650)


# 2. Какой из фильмов самый длительный (в минутах)?

In [11]:
answer_2 = data[data.runtime == data.runtime.max()]

get_answer_form_df(answer_2)

'Gods and Generals (tt0279111)'

In [12]:
answers['2'] = 'Gods and Generals (tt0279111)'

# 3. Какой из фильмов самый короткий (в минутах)?





In [13]:
answer_3 = data[data.runtime == data.runtime.min()]

get_answer_form_df(answer_3)

'Winnie the Pooh (tt1449283)'

In [14]:
answers['3'] = 'Winnie the Pooh (tt1449283)'

# 4. Какова средняя длительность фильмов?


In [15]:
answers['4'] = data.runtime.mean()

print(round(answers['4']))

110.0


# 5. Каково медианное значение длительности фильмов? 

In [16]:
answers['5'] = data.runtime.median()

print(answers['5'])

107.0


# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [17]:
# лучше код получения столбца profit вынести в Предобработку что в начале

In [18]:
answer_6 = new_data[new_data.profit == new_data.profit.max()]

get_answer_form_df(answer_6)

'Avatar (tt0499549)'

Второй вариант

In [19]:
answers['6'] = 'Avatar (tt0499549)'

In [20]:
display(new_data.sort_values(['profit'], ascending=False).head(1))

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit
239,tt0499549,237000000,2781505847,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weave...",[James Cameron],Enter the World of Pandora.,"In the 22nd century, a paraplegic Marine is di...",162,"[Action, Adventure, Fantasy, Science Fiction]","[Ingenious Film Partners, Twentieth Century Fo...",12/10/2009,7.1,2009,2544505847


# 7. Какой фильм самый убыточный? 

In [21]:
answer_7 = new_data[new_data.profit == new_data.profit.min()]

get_answer_form_df(answer_7)

'The Lone Ranger (tt1210819)'

In [22]:
answers['7'] = 'The Lone Ranger (tt1210819)'

Второй вариант

In [23]:
display(new_data.sort_values(['profit']).head(1))

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit
1245,tt1210819,255000000,89289910,The Lone Ranger,"[Johnny Depp, Armie Hammer, William Fichtner, ...",[Gore Verbinski],Never Take Off the Mask,The Texas Rangers chase down a gang of outlaws...,149,"[Action, Adventure, Western]","[Walt Disney Pictures, Jerry Bruckheimer Films...",7/3/2013,6.0,2013,-165710090


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [24]:
len(new_data[new_data.revenue > new_data.budget])

1478

In [25]:
new_data.query('revenue > budget')['imdb_id'].count()

1478

In [26]:
answers['8'] = 1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [27]:
answer_9 = new_data.query('release_year == 2008')\
    .sort_values(['revenue'], ascending=False).head(1)

get_answer_form_df(answer_9)

'The Dark Knight (tt0468569)'

In [28]:
answers['9'] = 'The Dark Knight (tt0468569)'

Второй вариант

In [29]:
data_2008 = new_data[new_data.release_year == 2008]

data_2008[data_2008.revenue == data_2008.revenue.max()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit
599,tt0468569,185000000,1001921825,The Dark Knight,"[Christian Bale, Michael Caine, Heath Ledger, ...",[Christopher Nolan],Why So Serious?,Batman raises the stakes in his war on crime. ...,152,"[Drama, Action, Crime, Thriller]","[DC Comics, Legendary Pictures, Warner Bros., ...",7/16/2008,8.1,2008,816921825


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [30]:
answer_10 = new_data.query('release_year >= 2012 & release_year <= 2014')\
                    .sort_values(['profit']).head(1)

get_answer_form_df(answer_10)


'The Lone Ranger (tt1210819)'

In [31]:
answers['10'] = 'The Lone Ranger (tt1210819)'

Второй вариант

In [32]:
data_2012_2014 = new_data[(new_data.release_year >= 2012) & (new_data.release_year <= 2014)]

data_2012_2014.loc[data_2012_2014.profit.idxmin()]['original_title']

'The Lone Ranger'

# 11. Какого жанра фильмов больше всего?

In [33]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале

In [34]:
data_with_genres = new_data.explode('genres')

data_with_genres['genres'].value_counts().idxmax()

'Drama'

In [35]:
answers['11'] = 'Drama'

ВАРИАНТ 2

In [36]:
genres = {}
list_of_genres = []

for index in new_data.index:
    list_of_genres = new_data.loc[index]['genres']
    for i, genre in enumerate(list_of_genres):
        if genre in list(genres.keys()):
            genres[genre] += 1
        else:
            genres[genre] = 1

df_genres = pd.Series(data=genres, index=list(genres.keys()))

df_genres.sort_values(ascending=False).head(1)

Drama    782
dtype: int64

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [37]:
profit_movies = new_data.query('profit > 0').copy()

pivot_table_with_profit = pivot_table_from_data(
    profit_movies, 'imdb_id', 'genres', 'profit')

display(pivot_table_with_profit.count().sort_values(ascending=False).head(1))

Drama    560
dtype: int64

In [38]:
data_with_genres.query('profit > 0')['genres'].describe(include=object).top

'Drama'

In [39]:
answers['12'] = 'Drama'


# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [40]:
pivot_table_with_directors = pivot_table_from_data(new_data, 'imdb_id', 'director', 'revenue')

pivot_table_with_directors.sum().idxmax()

'Peter Jackson'

In [41]:
data_with_directors = new_data.explode('director')

# группируем по режиссерам, суммируя сборы и, сортирую по возрастанию, берём последнего
data_with_directors.groupby('director')['revenue'].sum().sort_values().tail(1)

director
Peter Jackson    6490593685
Name: revenue, dtype: int64

In [42]:
answers['13'] = 'Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [43]:
# разбить список жанров у каждого режиссера построчно
data_with_directors_and_genres = data_with_directors.explode('genres')

# оствляем только Action, группируем записи по режиссерам и считаем кол-во фильмов у каждого
data_with_directors_and_genres.query('genres == "Action"').groupby('director')[
    'imdb_id'].count().idxmax()

'Robert Rodriguez'

In [44]:
# другой вариант, считаем по исходному датасету
# собираем список режиссеров (может быть больше одного), у которых есть фильмы Action
t_directors_with_action = data[data.genres.str.contains('Action')]['director'].copy()

# разбиваем списки режиссеров построчно
directors_with_action = string_to_list(t_directors_with_action).explode()

# считаем, сколько на каждого режиссера приходится упоминаний и берём максимальное
directors_with_action.value_counts().idxmax()

'Robert Rodriguez'

In [45]:
answers['14'] = 'Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [46]:
# оставим фильмы только 2012 года
new_data_2012 = new_data[new_data.release_year == 2012].copy()

# разобьём списки актёров построчно, сгруппируем по актёру 
# при групировке сложим сборы по всем фильмам и найдём максимальную сумму
new_data_2012.explode('cast').groupby('cast')['revenue'].sum().idxmax()

'Chris Hemsworth'

In [47]:
data_2012 = new_data.query('release_year == 2012').copy()

pivot_table_from_data(data_2012, 'imdb_id', 'cast', 'revenue').sum().idxmax()

'Chris Hemsworth'

In [48]:
answers['15'] = 'Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [49]:
# соберем список высокобюджетных фильмов: бюджет выше среднего
high_budget_movies = new_data[new_data.budget > new_data.budget.mean()].copy()

# разберём списки актёров построчно
cast_list = high_budget_movies[['imdb_id','cast']].explode('cast')

# сгруппируем записи по актёрам и посчитаем сколько на каждого пришлось фильмов
cast_list.groupby('cast')['imdb_id'].count().idxmax()

'Matt Damon'

In [50]:
answers['16'] = 'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [51]:
# оставим в датасете фильмы с Николасом Кейджем и вытащим стобец с жанрами
nc_genres = data[data.cast.str.contains('Nicolas Cage')]['genres']

# превратим строки в списки жанров и разобьём построчно каждый список
all_nc_genres = string_to_list(nc_genres).explode()

# считаем кол-во по каждому жанру и выбираем максимальное
all_nc_genres.value_counts().idxmax()

'Action'

In [52]:
# воспользуемся ранее собранным датафреймом

filmes_with_Cage = data_with_genres.explode('cast').query('cast == "Nicolas Cage"')

filmes_with_Cage.groupby('genres')['imdb_id'].count().sort_values().tail(1)

genres
Action    17
Name: imdb_id, dtype: int64

In [53]:
answers['17'] = 'Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [54]:
# разбиваем на отдельные строки список компаний в исходных записях
new_data_by_companies = new_data.explode('production_companies')

# отбираем фильмы Paramount Pictures
pp_movies = new_data_by_companies[
    new_data_by_companies.production_companies.str.contains('Paramount Pictures')]

pp_low_index = pp_movies['profit'].idxmin()

print(new_data.loc[pp_low_index]['original_title'] +
      ' (' + new_data.loc[pp_low_index]['imdb_id'] + ')')

K-19: The Widowmaker (tt0267626)


In [55]:
answers['18'] = 'K-19: The Widowmaker (tt0267626)'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [56]:
# собираем сводную таблицу: строки - фильмы, столбцы - года, на пересечении - сборы
# суммируем по строкам и столбцам, получаем строку итогов по каждому году
pivot_table_with_revenue = data.pivot_table(values='revenue', \
                        index=['imdb_id'], columns=['release_year'], aggfunc='sum', margins=True)

#  сортируем по возрастанию, нам нужно предпоследнее значение
pivot_table_with_revenue.loc['All'].sort_values().head(-1).idxmax()

2015

In [57]:
answers['19'] = 2015

# 20. Какой самый прибыльный год для студии Warner Bros?

In [58]:
#  собираем датафрейм из фильмов Warner Bros на исходном датасете
wb_movies = data[data.production_companies.str.contains("Warner Bros")].copy()

# добавляем колонку с прибылью
wb_movies['profit'] = wb_movies['revenue'] - wb_movies['budget']

# соираем сводную таблицу по прибыли за каждый код по каждому фильму и суммируем, выводя итоги
pivot_with_wb_movies = wb_movies.pivot_table(
    values='profit', index=['imdb_id'], columns=['release_year'], aggfunc='sum', margins=True)

# наш ответ в строке итогов, отсортированной по возрастанию, без последнего элемента (сумма за все годы)
pivot_with_wb_movies.loc['All'].sort_values().head(-1).idxmax()

2014

In [59]:
wb_movies.groupby('release_year')['profit'].sum().idxmax()

2014

In [60]:
answers['20'] = 2014

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [61]:
# записываем в новую колонку месяц выхода фильма
new_data['month'] = new_data.release_date.apply(get_month)

data_by_month = new_data.groupby(['month'])['imdb_id'].count()

month_dict[data_by_month.idxmax()]

'Сентябрь'

In [62]:
answers['21'] = 'Сентябрь'

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [63]:
answers['22'] = new_data.query('month in (6, 7, 8)').imdb_id.count()

print(answers['22'])

450


# 23. Для какого режиссера зима – самое продуктивное время года? 

In [64]:
directors_by_season = new_data[['director', 'release_date']].explode('director')

directors_by_season['season'] = directors_by_season.release_date.apply(get_season)

display(directors_by_season.query('season == "Зима"')['director'].value_counts())

Peter Jackson            7
Clint Eastwood           6
Steven Soderbergh        6
Shawn Levy               4
Nancy Meyers             4
                        ..
Sam Raimi                1
Richard Eyre             1
Diane Keaton             1
Christopher B. Landon    1
Jay Chandrasekhar        1
Name: director, Length: 358, dtype: int64

In [65]:
seasons = directors_by_season.pivot_table(values='release_date',\
                                index=['director'], columns=['season'], aggfunc='count', fill_value=0)

display(seasons.sample(10))

seasons['Зима'].idxmax()



season,Весна,Зима,Лето,Осень
director,,,,
David Yates,0,0,3,1
"George Tillman, Jr.",1,0,0,1
Scott Cooper,0,2,0,1
Joseph Ruben,0,0,0,1
Joshua Michael Stern,0,0,2,0
Adam Shankman,2,4,2,0
Dennie Gordon,1,0,0,0
Ron Shelton,0,1,0,0
Mark Dindal,0,1,0,1


'Peter Jackson'

In [66]:
answers['23'] = 'Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [67]:
# добавляем колонку с длиной символов в описании
new_data['title_length'] = new_data.original_title.apply(len)

display(new_data[['production_companies', 'title_length']].sample(3))

data_by_prod_companies = new_data.explode('production_companies')

data_by_prod_companies.groupby(['production_companies'])['title_length'].mean().idxmax()

,production_companies,title_length
879,"[DNA Films, British Film Council]",16
1169,"[Splendid Pictures, Maverick Films, Metro-Gold...",16
471,"[GK Films, Henceforth, Projection Pictures, Lo...",16


'Four By Two Productions'

In [68]:
answers['24'] = 'Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [69]:
# добавляем столбец с кол-вом слов в описании фильма
new_data['len_overview'] = new_data.overview.apply(str.split).apply(len)

data_by_prod_companies = new_data.explode('production_companies')

answers['25'] = data_by_prod_companies.groupby(['production_companies'])['len_overview'].mean().idxmax()

answers['25']

'Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [70]:
vote_99 = new_data.vote_average.quantile(q=0.99)

print(vote_99)

7.8


In [71]:
top_movies = new_data[new_data.vote_average > vote_99][[
    'imdb_id', 'original_title', 'vote_average']].copy()

top_movies.sort_values('original_title')

,imdb_id,original_title,vote_average
1191,tt2024544,12 Years a Slave,7.9
128,tt2267998,Gone Girl,7.9
119,tt2015381,Guardians of the Galaxy,7.9
370,tt1375666,Inception,7.9
9,tt2096673,Inside Out,8.0
118,tt0816692,Interstellar,8.0
1800,tt0209144,Memento,7.9
34,tt3170832,Room,8.0
599,tt0468569,The Dark Knight,8.1
138,tt2278388,The Grand Budapest Hotel,7.9


In [72]:
answers['26'] = top_movies['original_title'].values

answers['26']

array(['Inside Out', 'Room', 'Interstellar', 'Guardians of the Galaxy',
       'The Imitation Game', 'Gone Girl', 'The Grand Budapest Hotel',
       'Inception', 'The Dark Knight', 'The Pianist',
       'The Lord of the Rings: The Return of the King',
       'The Wolf of Wall Street', '12 Years a Slave', 'Memento'],
      dtype=object)

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [73]:
import itertools as it


def make_couples(cast):
    """Собираем элементы из переданного списка cast в пары"""
    return list(it.combinations(cast, 2))


# вытаскиваем столбец с актёрами из датасета
actors_only = new_data.cast.copy()

# собираем пары внутри каждого фильм и разбиваем построчно
actors_in_couples = actors_only.apply(make_couples).explode()

# считаем кол-во записей/фильмов для каждой пары и находим самое популярное
# на самом деле две пары подходят, но в тесте есть только один из двух вариантов
display(actors_in_couples.value_counts().head(2))

# забираем в словарь ответов первый найденный вариант
answers['27'] = actors_in_couples.value_counts().idxmax()

(Daniel Radcliffe, Rupert Grint)    8
(Daniel Radcliffe, Emma Watson)     8
Name: cast, dtype: int64

ВАРИАНТ 2 (проверка + список фильмов для выбранной пары)

In [74]:
# собираем датафрейм с актёрами и фильмами, разбиваем на пары построчно
df_cast = new_data[['cast', 'original_title']].copy()
df_cast.cast = df_cast.cast.apply(make_couples)
df_cast_in_pairs = df_cast.explode('cast')

# смотрим статику по df_cast_in_pairs
display(df_cast_in_pairs.describe(include=object)['cast'])

# список фильмов, в которых снялась наша пара
display(df_cast_in_pairs[df_cast_in_pairs.cast == answers['27']])

count                                18868
unique                               18121
top       (Daniel Radcliffe, Rupert Grint)
freq                                     8
Name: cast, dtype: object

,cast,original_title
242,"(Daniel Radcliffe, Rupert Grint)",Harry Potter and the Half-Blood Prince
374,"(Daniel Radcliffe, Rupert Grint)",Harry Potter and the Deathly Hallows: Part 1
497,"(Daniel Radcliffe, Rupert Grint)",Harry Potter and the Philosopher's Stone
722,"(Daniel Radcliffe, Rupert Grint)",Harry Potter and the Deathly Hallows: Part 2
864,"(Daniel Radcliffe, Rupert Grint)",Harry Potter and the Chamber of Secrets
1312,"(Daniel Radcliffe, Rupert Grint)",Harry Potter and the Goblet of Fire
1560,"(Daniel Radcliffe, Rupert Grint)",Harry Potter and the Prisoner of Azkaban
1670,"(Daniel Radcliffe, Rupert Grint)",Harry Potter and the Order of the Phoenix


# Submission

In [75]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 109.6585494970884,
 '5': 107.0,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': 2015,
 '20': 2014,
 '21': 'Сентябрь',
 '22': 450,
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': array(['Inside Out', 'Room', 'Interstellar', 'Guardians of the Galaxy',
        'The Imitation Game', 'Gone Girl', 'The Grand Budapest Hotel',
        'Inception', 'The Dark Knight', 'The Pianist',
        'The Lord of the Rings: The Return of the King',
        'The Wolf of Wall Street', '12 Years a Slave', 'Memento'],
   

In [76]:
# и убедиться что ни чего не пропустил)
len(answers)

27

Все ответы в тесте оказались правильными.